<a href="https://colab.research.google.com/github/koushikroy/face_recognition/blob/main/face_recognition_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setups and Installs

In [4]:
!pip install mediapipe

In [5]:
!git clone https://github.com/koushikroy/face_recognition.git

fatal: destination path 'face_recognition' already exists and is not an empty directory.


# Data Processing and Database making

## Reading face images from the directory

In [7]:
import os
face_database_paths = []
for file in os.listdir("/content/face_recognition/face_database"):
    if file.endswith(".png"):
        face_database_paths.append("/content/face_recognition/face_database/"+ file)

In [8]:
face_database_paths

['/content/face_recognition/face_database/thor.png',
 '/content/face_recognition/face_database/warmachine.png',
 '/content/face_recognition/face_database/captain_america.png',
 '/content/face_recognition/face_database/captain_marvel.png',
 '/content/face_recognition/face_database/gamora.png',
 '/content/face_recognition/face_database/hulk.png',
 '/content/face_recognition/face_database/ironman.png',
 '/content/face_recognition/face_database/spiderman.png',
 '/content/face_recognition/face_database/antman.png',
 '/content/face_recognition/face_database/black_panther.png',
 '/content/face_recognition/face_database/wasp.png',
 '/content/face_recognition/face_database/hawkeye.png',
 '/content/face_recognition/face_database/strange.png',
 '/content/face_recognition/face_database/wanda.png',
 '/content/face_recognition/face_database/nebula.png',
 '/content/face_recognition/face_database/black_widow.png']

## Applying mediapipe and extracting the face landmarks

In [9]:
# Creating pandas dataframe for storing the landmark points
import pandas as pd

# initializing column names
columns = []
for i in range (478):
    columns.append(str(i)+'x')
    columns.append(str(i)+'y')
    columns.append(str(i)+'z')
columns.append('output')

# Defining empty dataframe
df = pd.DataFrame(columns=columns)
df.head()

,0x,0y,0z,1x,1y,1z,2x,2y,2z,3x,3y,3z,4x,4y,4z,5x,5y,5z,6x,6y,6z,7x,7y,7z,8x,8y,8z,9x,9y,9z,10x,10y,10z,11x,11y,11z,12x,12y,12z,13x,...,465x,465y,465z,466x,466y,466z,467x,467y,467z,468x,468y,468z,469x,469y,469z,470x,470y,470z,471x,471y,471z,472x,472y,472z,473x,473y,473z,474x,474y,474z,475x,475y,475z,476x,476y,476z,477x,477y,477z,output


In [10]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

# For static images:
IMAGE_FILES = face_database_paths
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5) as face_mesh:
  for idx, file in enumerate(IMAGE_FILES):
    image = cv2.imread(file)
    # Convert the BGR image to RGB before processing.
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Print and draw face mesh landmarks on the image.
    if not results.multi_face_landmarks:
      continue
    face_landmarks = results.multi_face_landmarks[0]
    landmark_list = []
    for i in range(478):
      landmark_list.append(face_landmarks.landmark[i].x)
      landmark_list.append(face_landmarks.landmark[i].y)
      landmark_list.append(face_landmarks.landmark[i].z)
    landmark_list.append(file.split('/')[-1].split('.')[0])
    df.loc[idx] = landmark_list


In [12]:
df.to_csv('marvel_faces.csv', index = False)

## Calculating distance from the landmark points

In [3]:
import pandas as pd
face_gt = pd.read_csv('/content/face_recognition/csv_files/marvel_faces.csv')

In [27]:
dis_col_names = []
for i in range(478):
    for j in range(i+1,478):
        dis_col_names.append(str(i)+'-'+str(j))

In [28]:
df_distance = pd.DataFrame(columns = dis_col_names)

In [12]:
# dis_col_names[0].split('-')[0], dis_col_names[0].split('-')[1]

('0', '1')

In [29]:
from scipy.spatial import distance

for i in range(face_gt.shape[0]):
    dis_list = []
    for each_name in dis_col_names:
        a = (face_gt[each_name.split('-')[0]+'x'][i], 
             face_gt[each_name.split('-')[0]+'y'][i], 
             face_gt[each_name.split('-')[0]+'z'][i])
        b = (face_gt[each_name.split('-')[1]+'x'][i], 
             face_gt[each_name.split('-')[1]+'y'][i], 
             face_gt[each_name.split('-')[1]+'z'][i])
        dis = distance.euclidean(a, b)
        dis_list.append(dis)
    df_distance.loc[i] = dis_list


In [31]:
df_distance['output'] = face_gt['output']

In [32]:
df_distance

,0-1,0-2,0-3,0-4,0-5,0-6,0-7,0-8,0-9,0-10,0-11,0-12,0-13,0-14,0-15,0-16,0-17,0-18,0-19,0-20,0-21,0-22,0-23,0-24,0-25,0-26,0-27,0-28,0-29,0-30,0-31,0-32,0-33,0-34,0-35,0-36,0-37,0-38,0-39,0-40,...,468-475,468-476,468-477,469-470,469-471,469-472,469-473,469-474,469-475,469-476,469-477,470-471,470-472,470-473,470-474,470-475,470-476,470-477,471-472,471-473,471-474,471-475,471-476,471-477,472-473,472-474,472-475,472-476,472-477,473-474,473-475,473-476,473-477,474-475,474-476,474-477,475-476,475-477,476-477,output
0,0.050030,0.027876,0.074124,0.060521,0.069124,0.091850,0.139578,0.111312,0.123409,0.175025,0.003521,0.008495,0.015435,0.015780,0.017444,0.019182,0.024760,0.043838,0.042185,0.038018,0.219637,0.111204,0.116586,0.124184,0.140905,0.108523,0.138917,0.131186,0.145315,0.148598,0.146052,0.074284,0.145236,0.202201,0.165342,0.082013,0.013926,0.014945,0.027385,0.039127,...,0.125220,0.113622,0.127434,0.015383,0.023772,0.013055,0.114374,0.126748,0.113474,0.101907,0.115753,0.012988,0.015689,0.128131,0.140509,0.126776,0.115646,0.129897,0.015347,0.137961,0.150362,0.137076,0.125457,0.139239,0.124708,0.137067,0.124296,0.112268,0.125569,0.012467,0.007761,0.012587,0.007686,0.015685,0.025054,0.013599,0.013635,0.015448,0.015794,thor
1,0.048130,0.026766,0.084789,0.061721,0.075550,0.113651,0.145917,0.149281,0.167650,0.231878,0.005061,0.010307,0.014417,0.040939,0.045233,0.051099,0.058215,0.075051,0.038785,0.035663,0.226201,0.121035,0.124733,0.131109,0.145292,0.121435,0.157181,0.152534,0.160343,0.160320,0.147215,0.103335,0.151033,0.192888,0.164660,0.081642,0.016927,0.018786,0.033893,0.048230,...,0.115866,0.103412,0.114674,0.014943,0.021285,0.014959,0.104242,0.115584,0.105486,0.092976,0.104222,0.015113,0.020998,0.114777,0.126170,0.114909,0.103440,0.115755,0.014784,0.125296,0.136680,0.126375,0.113970,0.125244,0.115690,0.126945,0.117820,0.104528,0.114658,0.011501,0.011126,0.011469,0.011023,0.016147,0.022970,0.015750,0.015803,0.022148,0.016115,warmachine
2,0.070448,0.036918,0.111124,0.087408,0.102525,0.145431,0.206935,0.175769,0.195459,0.285305,0.004833,0.010889,0.018651,0.043332,0.048054,0.054683,0.064447,0.086037,0.058053,0.051150,0.325228,0.169382,0.176016,0.185991,0.208350,0.166528,0.205100,0.196387,0.213084,0.217740,0.216213,0.137941,0.214143,0.296098,0.242742,0.119518,0.024095,0.025580,0.048823,0.070104,...,0.189940,0.172137,0.186411,0.022263,0.036694,0.022597,0.169814,0.185646,0.171820,0.154083,0.168438,0.022575,0.025698,0.188917,0.204781,0.190171,0.173130,0.188213,0.022157,0.206140,0.222012,0.208208,0.190358,0.204564,0.187960,0.203737,0.190772,0.172303,0.185694,0.015992,0.011670,0.015934,0.011534,0.018799,0.031925,0.020792,0.020597,0.023204,0.018648,captain_america
3,0.066451,0.033512,0.095748,0.080635,0.091878,0.120081,0.171547,0.150564,0.168246,0.236867,0.004453,0.010405,0.018055,0.020423,0.022074,0.025022,0.032118,0.053282,0.055642,0.047212,0.262164,0.134586,0.142045,0.152372,0.173308,0.131481,0.173172,0.163159,0.181099,0.184651,0.179686,0.100988,0.178907,0.244431,0.201126,0.102758,0.020005,0.021728,0.038931,0.056450,...,0.161408,0.145898,0.162509,0.019387,0.032028,0.018960,0.145871,0.161757,0.145719,0.130213,0.146799,0.018932,0.021136,0.162191,0.178071,0.161290,0.146527,0.163783,0.019461,0.177497,0.193452,0.177254,0.161747,0.178375,0.161800,0.177709,0.162302,0.146116,0.161996,0.016108,0.010749,0.015931,0.010776,0.019690,0.032039,0.019064,0.018829,0.021525,0.019601,captain_marvel
4,0.055329,0.023264,0.075217,0.066435,0.072442,0.094188,0.172096,0.127706,0.143583,0.200507,0.004972,0.011353,0.018013,0.025581,0.028054,0.032302,0.039037,0.052678,0.044856,0.033020,0.291843,0.130828,0.142079,0.155650,0.179207,0.124267,0.168838,0.156268,0.180398,0.187648,0.191920,0.096050,0.179815,0.270985,0.219054,0.106211,0.023026,0.024300,0.048322,0.067243,...,0.168780,0.158119,0.171617,0.017817,0.029039,0.016524,0.157423,0.168917,0.156140,0.145661,0.158941,0.017012,0.019427,0.171342,0.182981,0.169500,0.159384,0.173352,0.018472,0.183999,0.195813,0.182634,0.1718

In [33]:
df_distance.to_csv('df_distance.csv', index = False)

# Face Recognition: Method 1

## Load and Normalize the dataset

In [ ]:
# Import the distance csv and unzip it
!unrar x "/content/face_recognition/csv_files/df_distance.rar" "/content/"

In [7]:
import pandas as pd
df_dist = pd.read_csv('df_distance.csv')

In [9]:
import numpy as np

In [44]:
max_list = df_dist.max(axis = 1)
df_dist_norm = df_dist.drop('output', axis = 1)
for i in range(df_dist.shape[0]):
    df_dist_norm.loc[i] = df_dist_norm.loc[i]/max_list[i]

In [50]:
df_dist_norm.max(axis=1)

0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     1.0
8     1.0
9     1.0
10    1.0
11    1.0
12    1.0
13    1.0
14    1.0
dtype: float64

## Load Image and Compare with the dataset

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))